# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

# Function to create pandas Data Frame
def pandas_factory (colnames, rows):
    return pd.DataFrame(rows, columns = colnames)
        

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of denormalized data in the <font color=red>**event_datafile_new.csv**</font> :<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [7]:
# Makingconnection to a Cassandra instance on local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace 
session.execute('''create keyspace if not exists sparkify
                with replication = {'class': 'SimpleStrategy', 'replication_factor': 1}''')


#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Queries:

### 1. Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query1: Get the artist, song title and song's length in the music app history that was heard during \
### sessionId = 338, and itemInSession = 4

In [10]:
try:
    session.execute(''' CREATE TABLE IF NOT EXISTS song_sessions 
    (sessionId int, itemInSession int, artist text, song_title text, song_length float,
    PRIMARY KEY(sessionId, itemInSession))''')
except Exception as e:
    print(e)

                    

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_sessions (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "values(%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [12]:
session.row_factory = pandas_factory

result = session.execute(""" select artist, song_title, song_length 
                            from song_sessions                             
                            WHERE sessionId = 338 AND itemInSession = 4 """)

df = result._current_rows 
df

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2: Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

In [13]:
try:
    session.execute(''' CREATE TABLE IF NOT EXISTS song_user_session_info
    (user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, 
    PRIMARY KEY((user_id, session_id),item_in_session))''')
except Exception as e:
    print(e)

                    

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_session_info (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[0], line[-2], line[1], line[4]))
        

In [15]:
session.row_factory = pandas_factory

result = session.execute(""" select artist, song, first_name, last_name 
                            from song_user_session_info                             
                            WHERE user_id = 10 AND session_id = 182
                            ORDER BY item_in_session""")

df = result._current_rows 
df

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3: Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [18]:
try:
    session.execute(''' CREATE TABLE IF NOT EXISTS user_song_info
    (song text, first_name text, last_name text, user_id int, 
    PRIMARY KEY(song, first_name, last_name, user_id))''')
except Exception as e:
    print(e)
                    

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_info (song, first_name, last_name, user_id)"
        query = query + "values(%s, %s, %s, %s);"
        session.execute(query, (line[-2], line[1], line[4], int(line[-1])))
        

In [20]:
session.row_factory = pandas_factory

result = session.execute(""" select first_name, last_name 
                            from user_song_info                             
                            WHERE song = 'All Hands Against His Own'
                            """)

df = result._current_rows 
df

,first_name,last_name
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [21]:
# Drop the table before closing out the sessions

In [22]:
try:
    session.execute("DROP TABLE song_sessions")
    session.execute("DROP TABLE song_user_session_info")
    session.execute("DROP TABLE user_song_info")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()